In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# File path
file_path = '2022-09-01 415MHz RvsB 3.50K.txt'

# Load the file into a DataFrame
nfile = pd.read_csv(
    file_path,
    sep="\t", 
    header=0,
    na_values="NaN",
    #parse_dates=[['Date', 'Time']],
    #date_format='%Y/%m/%d %H:%M:%S',
    #dtype=float
)

nfile['Date'] = pd.to_datetime(nfile['Date'] + ' ' + nfile['Time'], format='%Y/%m/%d %H:%M:%S')
nfile.drop(columns=["Time"], inplace=True)
nfile.rename(columns={"Date":"Date_Time"}, inplace=True)

print(nfile.info())
print(nfile.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 38 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Date_Time                  15 non-null     datetime64[ns]
 1   Set Temp [K]               15 non-null     float64       
 2   Set Freq [Hz]              15 non-null     float64       
 3   Duty Cycle [%]             15 non-null     float64       
 4   Pulse Period [ms]          15 non-null     float64       
 5   P_forw (giga)              0 non-null      float64       
 6   P_refl (giga)              0 non-null      float64       
 7   P_trans (giga)             0 non-null      float64       
 8   CW Power (Tek)             15 non-null     float64       
 9   Pulse Power (Tek)          15 non-null     float64       
 10  Peak Power (Tek)           15 non-null     float64       
 11  DC meas [%] (Tek)          15 non-null     float64       
 12  P_trans fo

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming 'nfile' is the DataFrame from the combined .txt files
# Example data structure
# nfile = pd.read_csv("combined_output.csv")

# Define intervals for 'Peak Field on Sample [mT]'
intervals = [
    {"center": 20, "tolerance": 3},
    {"center": 30, "tolerance": 3},
    {"center": 40, "tolerance": 5},
    {"center": 50, "tolerance": 5}
]

results = []

# Calculate mean and standard deviation for each interval
for interval in intervals:
    lower_bound = interval["center"] - interval["tolerance"]
    upper_bound = interval["center"] + interval["tolerance"]
    
    # Filter data within the interval
    filtered_data = nfile[
        (nfile["Peak Field on Sample [mT]"] >= lower_bound) & 
        (nfile["Peak Field on Sample [mT]"] <= upper_bound)
    ]
    
    # Calculate mean and standard deviation
    mean_resistance = filtered_data["Surface Resistance [nOhm]"].mean()
    std_resistance = filtered_data["Surface Resistance [nOhm]"].std()
    
    # Append results
    results.append({
        "Interval": f"{interval['center']}±{interval['tolerance']}",
        "Mean Resistance (nOhm)": mean_resistance,
        "Std Resistance (nOhm)": std_resistance
    })

# Create a new DataFrame for results
results_df = pd.DataFrame(results)

# Save the results to a CSV file
results_df.to_csv("resistance_analysis.csv", index=False)

# Print the results
print(results_df)

# Plot the results
plt.errorbar(
    results_df["Interval"],
    results_df["Mean Resistance (nOhm)"],
    yerr=results_df["Std Resistance (nOhm)"],
    fmt='o',
    capsize=5,
    label="Surface Resistance"
)

plt.title("Surface Resistance Analysis")
plt.xlabel("Peak Field Interval [mT]")
plt.ylabel("Surface Resistance [nOhm]")
plt.grid(True)
plt.legend()
plt.savefig("resistance_analysis_plot.pdf", format="pdf", bbox_inches="tight")
plt.show()
